In [1]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('data.csv', lineterminator='\n')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212789 entries, 0 to 212788
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user            212789 non-null  int64 
 1   date            212789 non-null  object
 2   content         212789 non-null  object
 3   tweet_id        212789 non-null  int64 
 4   replyCount      212789 non-null  int64 
 5   retweetCount    212789 non-null  int64 
 6   likeCount       212789 non-null  int64 
 7   quoteCount      212789 non-null  int64 
 8   quotedTweet     35694 non-null   object
 9   mentionedUsers  91201 non-null   object
 10  hashtags        160869 non-null  object
dtypes: int64(6), object(5)
memory usage: 17.9+ MB


In [4]:
df.head()

,user,date,content,tweet_id,replyCount,retweetCount,likeCount,quoteCount,quotedTweet,mentionedUsers,hashtags
0,58010461,2021-11-04 23:58:13+00:00,To continually grow it audience &amp; make mor...,1456410488775061515,0,0,0,0,NaN,NaN,"['MONEY', 'socialmedia', 'censure', 'personal'..."
1,1393628979526094851,2021-11-04 23:57:39+00:00,My Linktr making it easier to follow my social...,1456410344138723333,0,0,3,0,NaN,NaN,"['smallstreamer', 'twitch', 'facebook', 'Youtu..."
2,617853906,2021-11-04 23:57:19+00:00,#giftsforher #tumblr #twitter #facebook #insta...,1456410261645058050,1,0,0,0,NaN,NaN,"['giftsforher', 'tumblr', 'twitter', 'facebook..."
3,617853906,2021-11-04 23:57:17+00:00,#linkedin #twitter #facebook #instagram #dogec...,1456410253952753665,1,0,0,0,NaN,NaN,"['linkedin', 'twitter', 'facebook', 'instagram..."
4,812704308089208832,2021-11-04 23:56:57+00:00,Quit Your Job With These 3 Winning #Facebook #...,1456410166157479941,0,1,1,0,NaN,NaN,"['Facebook', 'Marketing']"


In [5]:
df = df[['user', 'content']]

In [6]:
df.head()

,user,content
0,58010461,To continually grow it audience &amp; make mor...
1,1393628979526094851,My Linktr making it easier to follow my social...
2,617853906,#giftsforher #tumblr #twitter #facebook #insta...
3,617853906,#linkedin #twitter #facebook #instagram #dogec...
4,812704308089208832,Quit Your Job With These 3 Winning #Facebook #...


In [7]:
df['raw_tweet'] = df['content'].copy()

In [9]:
df.head()

,user,content,raw_tweet
0,58010461,To continually grow it audience &amp; make mor...,To continually grow it audience &amp; make mor...
1,1393628979526094851,My Linktr making it easier to follow my social...,My Linktr making it easier to follow my social...
2,617853906,#giftsforher #tumblr #twitter #facebook #insta...,#giftsforher #tumblr #twitter #facebook #insta...
3,617853906,#linkedin #twitter #facebook #instagram #dogec...,#linkedin #twitter #facebook #instagram #dogec...
4,812704308089208832,Quit Your Job With These 3 Winning #Facebook #...,Quit Your Job With These 3 Winning #Facebook #...


### Lowercase

In [10]:
df['content'] = df['content'].str.lower()

### Remove URLs

In [11]:
df['content'] = df['content'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df['content'] = df['content'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
df['content'] = df['content'].apply(lambda x: re.sub(r'bit.ly/\S+', '', x))
df['content'] = df['content'].apply(lambda x: re.sub(r'pic.twitter\S+','', x))

### Remove HTML references

In [12]:
df['content'] = df['content'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))

### Remove non-letter characters

In [13]:
df['content'] = df['content'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

### Remove users

In [14]:
df['content'] = df['content'].apply(lambda x: re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', x))
df['content'] = df['content'].apply(lambda x: re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', x))

### Remove hashtags

In [15]:
df['content'] = df['content'].apply(lambda x: re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', x))

### Remove Audio/Video tags

In [16]:
df['content'] = df['content'].apply(lambda x: re.sub('VIDEO:', '', x))
df['content'] = df['content'].apply(lambda x: re.sub('AUDIO:', '', x))

### Remove empty contents

In [17]:
df.replace("", np.nan, inplace=True)

In [18]:
df.dropna(subset = ["content"], inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212778 entries, 0 to 212788
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user       212778 non-null  int64 
 1   content    212778 non-null  object
 2   raw_tweet  212778 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.5+ MB


In [20]:
df.to_csv('data_preprocessed.csv')